## Import Libs

In [1]:
import os
import sys
import pickle
import pandas as pd
import numpy as np
from numpy import sort
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, auc, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from hyperopt import STATUS_OK, hp, fmin, tpe

In [2]:
# check the system font
import matplotlib.font_manager as font_manager
font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

# add the font wanted
font_dir = ['../../Latin-Modern-Roman/']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

# Set font family globally
plt.rcParams['font.family'] = 'Latin Modern Roman'
print(plt.rcParams['font.family'])

['Latin Modern Roman']


## Training Helper Functions

### Hyper-params Tuning

In [3]:
def BO_TPE(X_train, y_train, X_val, y_val):
    "Hyperparameter optimization"
    train = xgb.DMatrix(X_train, label=y_train)
    val = xgb.DMatrix(X_val, label=y_val)
    X_val_D = xgb.DMatrix(X_val)    #DMatrix is an internal data structure that is used by XGBoost, which is optimized for both memory efficiency and training speed.


    def objective(params):
        xgb_model = xgb.train(params, dtrain=train, num_boost_round=1000, evals=[(val, 'eval')],
                              verbose_eval=False, early_stopping_rounds=80)
        y_vd_pred = xgb_model.predict(X_val_D, ntree_limit=xgb_model.best_ntree_limit)
        y_val_class = [0 if i <= 0.5 else 1 for i in y_vd_pred]

        acc = accuracy_score(y_val, y_val_class)
        loss = 1 - acc

        return {'loss': loss, 'params': params, 'status': STATUS_OK}

    max_depths = [3, 4]
    learning_rates = [0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.15, 0.2]
    subsamples = [0.5, 0.6, 0.7, 0.8, 0.9]
    colsample_bytrees = [0.5, 0.6, 0.7, 0.8, 0.9]
    reg_alphas = [0.0, 0.005, 0.01, 0.05, 0.1]
    reg_lambdas = [0.8, 1, 1.5, 2, 4]

    space = {
        'max_depth': hp.choice('max_depth', max_depths),
        'learning_rate': hp.choice('learning_rate', learning_rates),
        'subsample': hp.choice('subsample', subsamples),
        'colsample_bytree': hp.choice('colsample_bytree', colsample_bytrees),
        'reg_alpha': hp.choice('reg_alpha', reg_alphas),
        'reg_lambda': hp.choice('reg_lambda', reg_lambdas),
    }

    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20)

    best_param = {'max_depth': max_depths[(best['max_depth'])],
                  'learning_rate': learning_rates[(best['learning_rate'])],
                  'subsample': subsamples[(best['subsample'])],
                  'colsample_bytree': colsample_bytrees[(best['colsample_bytree'])],
                  'reg_alpha': reg_alphas[(best['reg_alpha'])],
                  'reg_lambda': reg_lambdas[(best['reg_lambda'])]
                  }

    return best_param

### Train Model

In [4]:
def train_model(k, X_train, y_train, X_val, y_val, save_model_dir, fig_name_val, fig_name_fi, features):
    print('*************************************************************')
    print('{}th training ..............'.format(k + 1))
    print('Hyperparameters optimization')
    best_param = BO_TPE(X_train, y_train, X_val, y_val)
    print('Best Params: ', best_param)
    xgb_model = xgb.XGBClassifier(max_depth = best_param['max_depth'],
                                  eta = best_param['learning_rate'],
                                  n_estimators = 1000,
                                  subsample = best_param['subsample'],
                                  colsample_bytree = best_param['colsample_bytree'],
                                  reg_alpha = best_param['reg_alpha'],
                                  reg_lambda = best_param['reg_lambda'],
                                  objective = "binary:logistic"
                                  )

    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='error',
                  early_stopping_rounds=80, verbose=False)

    # print training performance
    y_tr_pred = (xgb_model.predict_proba(X_train, ntree_limit=xgb_model.best_ntree_limit))[:, 1]
    train_auc = roc_auc_score(y_train, y_tr_pred)
    print('training dataset AUC: ' + str(train_auc))
    y_tr_class = [0 if i <= 0.5 else 1 for i in y_tr_pred]
    acc = accuracy_score(y_train, y_tr_class)
    print('training dataset acc: ' + str(acc))

    #print validation performance 
    y_vd_pred = (xgb_model.predict_proba(X_val, ntree_limit=xgb_model.best_ntree_limit))[:, 1]
    valid_auc = roc_auc_score(y_val, y_vd_pred)
    print('validation dataset AUROC: ' + str(valid_auc))
    precision_, recall_, thresholds_ = precision_recall_curve(y_val, y_vd_pred)
    auprc_ = auc(recall_, precision_)
    print('validation dataset AUPRC: ' + str(auprc_))
    y_val_class = [0 if i <= 0.5 else 1 for i in y_vd_pred]
    valid_acc = accuracy_score(y_val, y_val_class)
    print('validation dataset acc: ' + str(valid_acc))
    f1_ = f1_score(y_val,y_val_class)
    print('validation dataset F1: ' + str(f1_))
    fpr_, tpr_, thresholds_ = roc_curve(y_val, y_vd_pred)
    print('************************************************************')

    # save the model
    save_model_path = save_model_dir + 'model{}.mdl'.format(k + 1)
    xgb_model.get_booster().save_model(fname=save_model_path)

    # Plot ROC,PRC and confusion matrix
    bc = BinaryClassification(y_val,y_vd_pred, labels=['Non-sepsis', 'Sepsis'])
    plt.figure(figsize=(20,20))
    plt.subplot2grid(shape=(2,4), loc=(0,0), colspan=2)
    bc.plot_roc_curve()
    plt.subplot2grid((2,4), (0,2), colspan=2)
    bc.plot_precision_recall_curve()
    plt.subplot2grid((2,4), (1,0), colspan=2)
    bc.plot_confusion_matrix()
    plt.subplot2grid((2,4), (1,2), colspan=2)
    bc.plot_confusion_matrix(normalize=True)
    plt.savefig('./figs/{}_{}.pdf'.format(fig_name_val, k+1))
    plt.clf()

    # < ----- uncomment below for feature selection process

    # # Explore feature importance 
    # print('---Feature Importance---')
    # # plot feature importance
    # plot_importance(xgb_model)

    # #plot graph of feature importances for better visualization
    # feat_importances = pd.Series(xgb_model.feature_importances_, index=features)
    # feat_importances.nlargest(len(features)).plot(kind='barh')
    # plt.savefig('./figs/{}_{}.pdf'.format(fig_name_fi, k+1))
    # plt.clf()


    # # Fit model using each importance as a threshold 
    # thresholds = sort(xgb_model.feature_importances_)
    # # feature and performance 
    # for thresh in thresholds:
    #     # select features using threshold
    #     selection = SelectFromModel(xgb_model, threshold=thresh, prefit=True)
    #     select_X_train = selection.transform(X_train)
    #     # train model
    #     selection_model = XGBClassifier()
    #     selection_model.fit(select_X_train, y_train)
    #     # eval model
    #     select_X_test = selection.transform(X_val)
    #     predictions = selection_model.predict(select_X_test)
    #     accuracy = accuracy_score(y_val, predictions)
    #     print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

### Data Process

In [5]:
def feature_extraction(case, data_features):
    
    labels = np.array(case['sepsis'])
    features = case[data_features]
    if 'time' in features.columns:
            features = features.drop(columns=['time'],axis = 1)

    return  features, labels

def data_process(data_set, data_path_dir, data_features):
    """
    Feature matrix across all patients in the data_set
    """
    frames_features = []
    frames_labels = []
    for csv in data_set:
        csv = csv.replace('psv','csv')
        patient = pd.read_csv(os.path.join(data_path_dir, csv), sep=',')
        features, labels = feature_extraction(patient, data_features)
        features = pd.DataFrame(features)
        labels = pd.DataFrame(labels)
        frames_features.append(features)
        frames_labels.append(labels)

    dat_features = np.array(pd.concat(frames_features))
    dat_labels = (np.array(pd.concat(frames_labels)))[:, 0]

    index = [i for i in range(len(dat_labels))]
    np.random.shuffle(index)
    dat_features = dat_features[index]
    dat_labels = dat_labels[index]

    return dat_features, dat_labels

In [6]:
def downsample(data_set, data_dir, data_features):
    """
    Using our feature extraction approach will result in over 1 million hours of data in the training process.
    However, only roughly 1.8% of these data corresponds to a positive outcome.
    Consequently, in order to deal with the serious class imbalance, a systematic way is provided by
    down sampling the excessive data instances of the majority class in each cross validation.
    """
    x, y = data_process(data_set, data_dir, data_features)
    index_0 = np.where(y == 0)[0]
    index_1 = np.where(y == 1)[0]

    index = index_0[len(index_1): -1]
    x_del = np.delete(x, index, 0)
    y_del = np.delete(y, index, 0)
    index = [i for i in range(len(y_del))]
    np.random.shuffle(index)
    x_del = x_del[index]
    y_del = y_del[index]

    return x_del, y_del

## Train Model

In [5]:
!mkdir -p ./trained_models/BDWOFS/
!mkdir -p ./trained_models/BDWFS/
!mkdir -p ./trained_models/EDWOFS/
!mkdir -p ./trained_models/EDWFS/

### Baseline Data without Feature Selection

In [7]:
X_feature_baseline = ['HR', 'SaO2', 'Temp', 'SBP', 'MAP', 'DBP', 'RR', 'BaseExcess', 'HCO3',
                      'PH', 'BUN', 'Calcium', 'Chloride', 'Creatinine', 'Glucose', 'Lactic',
                      'Magnesium', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age', 'gender']

y_feature = ['sepsis']

In [43]:
#load data from Cinc2019
data_path_c = "../../datasets/Cinc2019/baseline_all/"  
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')

# load data from MIMIC-III
data_path_m = "../../datasets/MIMICIII/adults/baseline_all/"  
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')

# 5-fold cross validation was implemented and five XGBoost models were produced
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))

    print('---Downsampling training data---')
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    x_train_c, y_train_c = downsample(train_set_c, data_path_c)

    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    x_train_m, y_train_m = downsample(train_set_m, data_path_m)

    x_train = np.concatenate((x_train_c, x_train_m))
    y_train = np.concatenate((y_train_c, y_train_m))

    print('---Downsampling validation data---')
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    x_val_c, y_val_c = downsample(val_set_c, data_path_c)

    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    x_val_m, y_val_m = downsample(val_set_m, data_path_m)

    x_val = np.concatenate((x_val_c, x_val_m))
    y_val = np.concatenate((y_val_c, y_val_m))

    print('---Start training process---')
    train_model(k, x_train, y_train, x_val, y_val, 
                save_model_dir = './trained_models/BDWOFS/',
                fig_name_val = 'val_wo_fs_baseline',
                fig_name_fi = 'feature_importance_baseline',
                features = X_feature_baseline)   #change here

---Fold1/5---
---Downsampling training data---
---Downsampling validation data---
---Start training process---
*************************************************************
1th training ..............
Hyperparameters optimization
  5%|▌         | 1/20 [00:51<16:20, 51.59s/trial, best loss: 0.3359514170040486]

/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/xgboost/core.py:93: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning



 10%|█         | 2/20 [01:28<12:55, 43.06s/trial, best loss: 0.3359514170040486]

/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/xgboost/core.py:93: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning



 15%|█▌        | 3/20 [04:25<29:27, 103.97s/trial, best loss: 0.3359514170040486]

...

<Figure size 432x288 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Baseline Data with Feature Engineering

From the results from above, with the increase of the feature numbers , accuracy starts to decreases when only 10 most important features are used. As a results, the most important 10 features, along with other 5 vital signs will be selected as the final 15 features.

In [8]:
X_feature_baseline_fs = ['Magnesium', 'HCO3', 'MAP', 'Chloride', 'SaO2', 'Potassium', 'BaseExcess', 'Glucose', 'PH', 'gender',
                        'RR', 'SBP', 'Temp', 'DBP', 'HR']

y_feature = ['sepsis']

In [56]:
#load data from Cinc2019
data_path_c = "../../datasets/Cinc2019/baseline_all/"  
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')

# load data from MIMIC-III
data_path_m = "../../datasets/MIMICIII/adults/baseline_all/"  
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')

# 5-fold cross validation was implemented and five XGBoost models were produced
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))

    print('---Downsampling training data---')
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    x_train_c, y_train_c = downsample(train_set_c, data_path_c, X_feature_baseline_fs)

    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    x_train_m, y_train_m = downsample(train_set_m, data_path_m, X_feature_baseline_fs)

    x_train = np.concatenate((x_train_c, x_train_m))
    y_train = np.concatenate((y_train_c, y_train_m))

    print('---Downsampling validation data---')
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    x_val_c, y_val_c = downsample(val_set_c, data_path_c, X_feature_baseline_fs)

    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    x_val_m, y_val_m = downsample(val_set_m, data_path_m, X_feature_baseline_fs)

    x_val = np.concatenate((x_val_c, x_val_m))
    y_val = np.concatenate((y_val_c, y_val_m))

    print('---Start training process---')
    train_model(k, x_train, y_train, x_val, y_val, 
                save_model_dir = './trained_models/BDWFS/',
                fig_name_val = 'val_w_fs_baseline',
                fig_name_fi = 'feature_importance_baseline_fs',
                features = X_feature_baseline_fs)   #change here

---Fold1/5---
---Downsampling training data---
---Downsampling validation data---
---Start training process---
*************************************************************
1th training ..............
Hyperparameters optimization
  5%|▌         | 1/20 [00:17<05:29, 17.36s/trial, best loss: 0.35773279352226717]

/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/xgboost/core.py:93: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning



 10%|█         | 2/20 [00:23<03:14, 10.83s/trial, best loss: 0.35773279352226717]

/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/xgboost/core.py:93: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning



 15%|█▌        | 3/20 [01:05<07:04, 24.98s/trial, best loss: 0.3567611336032389] 

...

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

### Engineered Data without Feature Engineering

In [9]:
X_feature_engineered = ['HR', 'SaO2', 'Temp', 'SBP', 'MAP', 'DBP', 'RR', 'BaseExcess', 'HCO3',
       'PH', 'BUN', 'Calcium', 'Chloride', 'Creatinine', 'Glucose', 'Lactic',
       'Magnesium', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age', 'gender',
       'HR_dev_1', 'HR_dev_2', 'HR_dev_3', 'RR_dev_1',
       'RR_dev_2', 'RR_dev_3', 'Temp_dev_1', 'Temp_dev_2', 'Temp_dev_3',
       'Bradycardia', 'Tachycardia', 'Hypothermia', 'Fever', 'Hyperpyrexia']

y_feature = ['sepsis']

In [45]:
#load data from Cinc2019
data_path_c = "../../datasets/Cinc2019/engineered_all/"  
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')

# load data from MIMIC-III
data_path_m = "../../datasets/MIMICIII/adults/engineered_all/"  
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')

# 5-fold cross validation was implemented and five XGBoost models were produced
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))

    print('---Downsampling training data---')
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    x_train_c, y_train_c = downsample(train_set_c, data_path_c, X_feature_engineered)

    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    x_train_m, y_train_m = downsample(train_set_m, data_path_m, X_feature_engineered)

    x_train = np.concatenate((x_train_c, x_train_m))
    y_train = np.concatenate((y_train_c, y_train_m))

    print('---Downsampling validation data---')
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    x_val_c, y_val_c = downsample(val_set_c, data_path_c, X_feature_engineered)

    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    x_val_m, y_val_m = downsample(val_set_m, data_path_m, X_feature_engineered)

    x_val = np.concatenate((x_val_c, x_val_m))
    y_val = np.concatenate((y_val_c, y_val_m))

    print('---Start training process---')
    train_model(k, x_train, y_train, x_val, y_val, 
                save_model_dir = './trained_models/EDWOFS/',
                fig_name_val = 'val_wo_fs_engineered',
                fig_name_fi = 'feature_importance_engineered',
                features = X_feature_engineered)   #change here

---Fold1/5---
---Downsampling training data---
---Downsampling validation data---
---Start training process---
*************************************************************
1th training ..............
Hyperparameters optimization
  5%|▌         | 1/20 [00:40<12:44, 40.24s/trial, best loss: 0.13506072874493924]

/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/xgboost/core.py:93: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning



 10%|█         | 2/20 [01:33<14:21, 47.85s/trial, best loss: 0.13506072874493924]

/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/xgboost/core.py:93: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning



 15%|█▌        | 3/20 [02:03<11:18, 39.91s/trial, best loss: 0.13506072874493924]

...

<Figure size 432x288 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Engineered Data with Feature Engineering

When use 9-17 most important features, the model has higher accuracy. As a results, 17 most important features + 4 extra vital signs  = 21 features

In [10]:
X_feature_engineered_fs = ['RR_dev_3', 'Temp_dev_3', 'SaO2', 'Magnesium', 'MAP', 
                            'gender', 'Potassium', 'Chloride', 'PTT', 'SBP', 
                            'HCO3', 'BaseExcess', 'Glucose', 'WBC', 'BUN', 
                            'Calcium', 'Platelet', 
                            'RR', 'Temp', 'DBP', 'HR']

y_feature = ['sepsis']

In [58]:
#load data from Cinc2019
data_path_c = "../../datasets/Cinc2019/engineered_all/"  
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')

# load data from MIMIC-III
data_path_m = "../../datasets/MIMICIII/adults/engineered_all/"  
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')

# 5-fold cross validation was implemented and five XGBoost models were produced
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))

    print('---Downsampling training data---')
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    x_train_c, y_train_c = downsample(train_set_c, data_path_c, X_feature_engineered_fs)

    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    x_train_m, y_train_m = downsample(train_set_m, data_path_m, X_feature_engineered_fs)

    x_train = np.concatenate((x_train_c, x_train_m))
    y_train = np.concatenate((y_train_c, y_train_m))

    print('---Downsampling validation data---')
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    x_val_c, y_val_c = downsample(val_set_c, data_path_c, X_feature_engineered_fs)

    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    x_val_m, y_val_m = downsample(val_set_m, data_path_m, X_feature_engineered_fs)

    x_val = np.concatenate((x_val_c, x_val_m))
    y_val = np.concatenate((y_val_c, y_val_m))

    print('---Start training process---')
    train_model(k, x_train, y_train, x_val, y_val, 
                save_model_dir = './trained_models/EDWFS/',
                fig_name_val = 'val_w_fs_engineered',
                fig_name_fi = 'feature_importance_engineered',
                features = X_feature_engineered_fs)   #change here

---Fold1/5---
---Downsampling training data---
---Downsampling validation data---
---Start training process---
*************************************************************
1th training ..............
Hyperparameters optimization
  5%|▌         | 1/20 [00:57<18:06, 57.19s/trial, best loss: 0.22024291497975712]

/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/xgboost/core.py:93: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning



 10%|█         | 2/20 [01:33<13:28, 44.90s/trial, best loss: 0.2191902834008097] 

/Users/jinyijin/.conda/envs/DL_Environment/lib/python3.7/site-packages/xgboost/core.py:93: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning



 15%|█▌        | 3/20 [01:44<08:21, 29.48s/trial, best loss: 0.2152226720647773]

...

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

## Performance on Test data

### Score Function

In [12]:
#!/usr/bin/env python

# This file contains functions for evaluating algorithms for the 2019 PhysioNet/
# CinC Challenge. You can run it as follows:
#
#   python evaluate_sepsis_score.py labels predictions scores.csv
#
# where 'labels' is a directory containing files with labels, 'predictions' is a
# directory containing files with predictions, and 'scores.csv' (optional) is a
# collection of scores for the predictions.

################################################################################

# The evaluate_scores function computes a normalized utility score for a cohort
# of patients along with several traditional scoring metrics.
#
# Inputs:
#   'label_directory' is a directory of pipe-delimited text files containing a
#   binary vector of labels for whether a patient is not septic (0) or septic
#   (1) for each time interval.
#
#   'prediction_directory' is a directory of pipe-delimited text files, where
#   the first column of the file gives the predicted probability that the
#   patient is septic at each time, and the second column of the file is a
#   binarized version of this vector. Note that there must be a prediction for
#   every label.
#
# Outputs:
#   'auroc' is the area under the receiver operating characteristic curve
#   (AUROC).
#
#   'auprc' is the area under the precision recall curve (AUPRC).
#
#   'accuracy' is accuracy.
#
#   'f_measure' is F-measure.
#
#   'normalized_observed_utility' is a normalized utility-based measure that we
#   created for the Challenge. This score is normalized so that a perfect score
#   is 1 and no positive predictions is 0.
#
# Example:
#   Omitted due to length. See the below examples.

import numpy as np, os, os.path, sys, warnings

def evaluate_sepsis_score(label_directory, prediction_directory):
    # Set parameters.
    label_header       = 'sepsis'
    prediction_header  = 'PredictedLabel'
    probability_header = 'PredictedProbability'

    dt_early   = -12
    dt_optimal = -6
    dt_late    = 3

    max_u_tp = 1
    min_u_fn = -2
    u_fp     = -0.05
    u_tn     = 0

    # Find label and prediction files.
    label_files = []
    for f in os.listdir(label_directory):
        g = os.path.join(label_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('csv'):
            label_files.append(g)
    label_files = sorted(label_files)

    prediction_files = []
    for f in os.listdir(prediction_directory):
        g = os.path.join(prediction_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('csv'):
            prediction_files.append(g)
    prediction_files = sorted(prediction_files)

    if len(label_files) != len(prediction_files):
        raise Exception('Numbers of label and prediction files must be the same.')

    # Load labels and predictions.
    num_files            = len(label_files)
    cohort_labels        = []
    cohort_predictions   = []
    cohort_probabilities = []

    for k in range(num_files):
        labels        = load_column(label_files[k], label_header, ',')
        predictions   = load_column(prediction_files[k], prediction_header, ',')
        probabilities = load_column(prediction_files[k], probability_header, ',')

        # Check labels and predictions for errors.
        if not (len(labels) == len(predictions) and len(predictions) == len(probabilities)):
            raise Exception('Numbers of labels and predictions for a file must be the same.')

        num_rows = len(labels)

        for i in range(num_rows):
            if labels[i] not in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

            if predictions[i] not in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

            if not 0 <= probabilities[i] <= 1:
                warnings.warn('Probabilities do not satisfy 0 <= probability <= 1.')

        if 0 < np.sum(predictions) < num_rows:
            min_probability_positive = np.min(probabilities[predictions == 1])
            max_probability_negative = np.max(probabilities[predictions == 0])

            if min_probability_positive <= max_probability_negative:
                warnings.warn('Predictions are inconsistent with probabilities, i.e., a positive prediction has a lower (or equal) probability than a negative prediction.')

        # Record labels and predictions.
        cohort_labels.append(labels)
        cohort_predictions.append(predictions)
        cohort_probabilities.append(probabilities)

    # Compute AUC, accuracy, and F-measure.
    labels        = np.concatenate(cohort_labels)
    predictions   = np.concatenate(cohort_predictions)
    probabilities = np.concatenate(cohort_probabilities)

    auroc, auprc        = compute_auc(labels, probabilities)
    accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

    # Compute utility.
    observed_utilities = np.zeros(num_files)
    best_utilities     = np.zeros(num_files)
    worst_utilities    = np.zeros(num_files)
    inaction_utilities = np.zeros(num_files)

    for k in range(num_files):
        labels = cohort_labels[k]
        num_rows          = len(labels)
        observed_predictions = cohort_predictions[k]
        best_predictions     = np.zeros(num_rows)
        worst_predictions    = np.zeros(num_rows)
        inaction_predictions = np.zeros(num_rows)

        if np.any(labels):
            t_sepsis = np.argmax(labels) - dt_optimal
            best_predictions[max(0, t_sepsis + dt_early) : min(t_sepsis + dt_late + 1, num_rows)] = 1
        worst_predictions = 1 - best_predictions

        observed_utilities[k] = compute_prediction_utility(labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        best_utilities[k]     = compute_prediction_utility(labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        worst_utilities[k]    = compute_prediction_utility(labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        inaction_utilities[k] = compute_prediction_utility(labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

    unnormalized_observed_utility = np.sum(observed_utilities)
    unnormalized_best_utility     = np.sum(best_utilities)
    unnormalized_worst_utility    = np.sum(worst_utilities)
    unnormalized_inaction_utility = np.sum(inaction_utilities)

    normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (unnormalized_best_utility - unnormalized_inaction_utility)

    return auroc, auprc, accuracy, f_measure, normalized_observed_utility

# The load_column function loads a column from a table.
#
# Inputs:
#   'filename' is a string containing a filename.
#
#   'header' is a string containing a header.
#
# Outputs:
#   'column' is a vector containing a column from the file with the given
#   header.
#
# Example:
#   Omitted.

def load_column(filename, header, delimiter):
    column = []
    with open(filename, 'r') as f:
        for i, l in enumerate(f):
            arrs = l.strip().split(delimiter)
            if i == 0:
                try:
                    j = arrs.index(header)
                except:
                    raise Exception('{} must contain column with header {} containing numerical entries.'.format(filename, header))
            else:
                if len(arrs[j]):
                    column.append(float(arrs[j]))
    return np.array(column)

# The compute_auc function computes AUROC and AUPRC as well as other summary
# statistics (TP, FP, FN, TN, TPR, TNR, PPV, NPV, etc.) that can be exposed
# from this function.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a probability vector, where predictions[i] gives the
#   predicted probability that the patient is septic at time i.  Note that there
#   must be a prediction for every label, i.e, len(labels) ==
#   len(predictions).
#
# Outputs:
#   'auroc' is a scalar that gives the AUROC of the algorithm using its
#   predicted probabilities, where specificity is interpolated for intermediate
#   sensitivity values.
#
#   'auprc' is a scalar that gives the AUPRC of the algorithm using its
#   predicted probabilities, where precision is a piecewise constant function of
#   recall.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0.3, 0.4, 0.6, 0.7, 0.8, 0.8]
#   In [3]: auroc, auprc = compute_auc(labels, predictions)
#   In [4]: auroc
#   Out[4]: 1.0
#   In [5]: auprc
#   Out[5]: 1.0

def compute_auc(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not 0 <= prediction <= 1:
                warnings.warn('Predictions do not satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds.
    thresholds = np.unique(predictions)[::-1]
    if thresholds[0] != 1:
        thresholds = np.insert(thresholds, 0, 1)
    if thresholds[-1] == 0:
        thresholds = thresholds[:-1]

    n = len(labels)
    m = len(thresholds)

    # Populate contingency table across prediction thresholds.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort the predicted probabilities from largest to
    # smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels)
            tn[j] = n - fn[j]
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest predicted probability.
        while i < n and predictions[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    # Compute AUROC as the area under a piecewise linear function with TPR /
    # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
    # under a piecewise constant with TPR / recall (x-axis) and PPV / precision
    # (y-axis).
    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr[j + 1] - tpr[j]) * ppv[j + 1]

    return auroc, auprc

# The compute_accuracy_f_measure function computes the accuracy and F-measure
# for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'accuracy' is a scalar that gives the accuracy of the predictions using its
#   binarized predictions.
#
#   'f_measure' is a scalar that gives the F-measure of the predictions using its
#   binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)
#   In [4]: accuracy
#   Out[4]: 0.666666666667
#   In [5]: f_measure
#   Out[5]: 0.666666666667

def compute_accuracy_f_measure(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    n = len(labels)
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif not labels[i] and predictions[i]:
            fp += 1
        elif labels[i] and not predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

# The compute_prediction_utility function computes the total time-dependent
# utility for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'utility' is a scalar that gives the total time-dependent utility of the
#   algorithm using its binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: utility = compute_prediction_utility(labels, predictions)
#   In [4]: utility
#   Out[4]: 3.388888888888889

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

        if dt_early >= dt_optimal:
            raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

        if dt_optimal >= dt_late:
            raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)

### Data Processing Functions

In [11]:
def save_challenge_predictions(file, scores, labels):
    with open(file, 'w') as f:
        f.write('PredictedProbability,PredictedLabel\n')
        for (s, l) in zip(scores, labels):
            f.write('%g,%d\n' % (s, l))

def save_challenge_testlabel(file, labels):
    with open(file, 'w') as f:
        f.write('sepsis\n')
        for l in labels:
            f.write('%d\n' % l)

def load_model_predict(X_test, k_fold, path):
    "ensemble the five XGBoost models by averaging their output probabilities"
    test_pred = np.zeros((X_test.shape[0], k_fold))
    X_test = xgb.DMatrix(X_test)
    for k in range(k_fold):
        # load the model
        model_path_name = path + 'model{}.mdl'.format(k+1) 
        loaded_model = xgb.Booster(model_file = model_path_name)
        # predict
        y_test_pred = loaded_model.predict(X_test)
        test_pred[:, k] = y_test_pred # save prediction results 5 times
    test_pred = pd.DataFrame(test_pred)
    result_pro = test_pred.mean(axis=1)

    return result_pro

def feature_extraction(case, data_features):
    labels = np.array(case['sepsis'])
    features = case[data_features]
    if 'time' in features.columns:
        features = features.drop(columns=['time'],axis = 1)

    return  features, labels    

def predict(data_set,
            data_dir,
            save_prediction_dir,
            save_label_dir,
            model_path,
            risk_threshold,
            data_features
            ):
    for csv in tqdm(data_set):
        csv = csv.replace('psv','csv')
        patient = pd.read_csv(data_dir+csv, sep=',')
        features, labels = feature_extraction(patient, data_features)

        predict_pro = load_model_predict(features, k_fold = 5, path = model_path)
        PredictedProbability = np.array(predict_pro)
        PredictedLabel = [0 if i <= risk_threshold else 1 for i in predict_pro]

        save_prediction_name = save_prediction_dir + csv
        save_challenge_predictions(save_prediction_name, PredictedProbability, PredictedLabel)
        save_testlabel_name = save_label_dir + csv
        save_challenge_testlabel(save_testlabel_name, labels)

### Baseline Data Without Feature Selection

In [62]:
!mkdir -p ./prediction
!mkdir -p ./label

In [67]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_baseline/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline) # change feature here 

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [00:41<00:00, 35.04it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.5740178764840992|0.11098545292100195|0.438724718039191|0.15921017857817044|0.48367491278822433


### Baseline Data with Feature Selection

In [68]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [69]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_baseline/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs) # change feature here 

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [00:39<00:00, 36.25it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.6815425885541813|0.16109074961611727|0.6605884947388319|0.21144162079868836|0.46946737003318306


### Engineered Data without Feature Selection

In [70]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [71]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_engineered/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered) # change feature here 

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [01:01<00:00, 23.65it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.91272257721576|0.4846181206970505|0.8033898305084746|0.4097306295399516|0.7709574151280525


### Engineered Data with Feature Selection

In [72]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [73]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_engineered/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs) # change feature here 

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [00:43<00:00, 32.98it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.875209880829091|0.49473153727386066|0.8121983491903472|0.3932744371615845|0.698775844465243


## Test Real Neonatal Data

### BDWOFS

In [12]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [13]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:08<00:00, 26.44it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.5474782172691877|0.04653345825087136|0.08378518553444292|0.0730776569356993|0.2674643732602103


### BDWFS

In [14]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [15]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:09<00:00, 25.02it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.5146374093704537|0.03954455230814661|0.03734263755952662|0.07191235965271149|0.2619095101582949


### EDWOFS

In [16]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [19]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:17<00:00, 13.04it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.9271443332455349|0.4126337929902422|0.586449148946202|0.1510018391249637|0.6612115168889443


### EDWFS

In [20]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [21]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:09<00:00, 23.53it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.7715935741981219|0.20554216403566714|0.04988448300249894|0.07219485243335329|0.2648219701690972


## Test Artificial Neonatal Data
### BDWOFS

In [13]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [14]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:10<00:00, 20.63it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.5054353455804399|0.03989395243360046|0.0771000587429019|0.07360817650238317|0.27011093107316475


### BDWFS

In [15]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [16]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:11<00:00, 19.12it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.6049068726333655|0.062246685574218634|0.052966516545917365|0.07532740655769046|0.2966512941958537


### EDWSFS

In [17]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [18]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:15<00:00, 14.47it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.905733081986732|0.4828730046619617|0.5315743097709027|0.13410551081350103|0.5773235281897878


### EDWFS

In [19]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [20]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:13<00:00, 16.66it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.8281642725711873|0.3580050186012843|0.2278735069512434|0.08652342618868361|0.3588724084922515
